In [ ]:
import numpy as np

from tqdm import tqdm, trange

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST

np.random.seed(0)
torch.manual_seed(0)
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn

In [ ]:
def patchify(images, n_patches):
    n, c, h, w = images.shape

    assert h == w, "Patchify method is implemented for square images only"

    patches = torch.zeros(n, n_patches ** 2, h * w * c // n_patches ** 2)
    patch_size = h // n_patches

    for idx, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:, i * patch_size: (i + 1) * patch_size, j * patch_size: (j + 1) * patch_size]
                patches[idx, i * n_patches + j] = patch.flatten()
    return patches

In [ ]:

class MyMSA(nn.Module):
    def __init__(self, d, n_heads=2):
        super(MyMSA, self).__init__()
        self.d = d
        self.n_heads = n_heads

        assert d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"

        d_head = int(d / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)

                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])

In [ ]:

class MyViTBlock(nn.Module):
    def __init__(self, hidden_d, n_heads, mlp_ratio=4):
        super(MyViTBlock, self).__init__()
        self.hidden_d = hidden_d
        self.n_heads = n_heads

        self.norm1 = nn.LayerNorm(hidden_d)
        self.mhsa = MyMSA(hidden_d, n_heads)
        self.norm2 = nn.LayerNorm(hidden_d)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, mlp_ratio * hidden_d),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_d, hidden_d)
        )

    def forward(self, x):
        out = x + self.mhsa(self.norm1(x))
        out = out + self.mlp(self.norm2(out))
        return out

In [ ]:

class MyViT(nn.Module):
    def __init__(self, chw, n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10):
        # Super constructor
        super(MyViT, self).__init__()

        # Attributes
        self.chw = chw # ( C , H , W )
        self.n_patches = n_patches
        self.n_blocks = n_blocks
        self.n_heads = n_heads
        self.hidden_d = hidden_d

        # Input and patches sizes
        assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)

        # 1) Linear mapper
        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

        # 2) Learnable classification token
        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))

        # 3) Positional embedding
        self.register_buffer('positional_embeddings', get_positional_embeddings(n_patches ** 2 + 1, hidden_d), persistent=False)

        # 4) Transformer encoder blocks
        self.blocks = nn.ModuleList([MyViTBlock(hidden_d, n_heads) for _ in range(n_blocks)])

        # 5) Classification MLPk
        self.mlp = nn.Sequential(
            nn.Linear(self.hidden_d, out_d),
            nn.Softmax(dim=-1)
        )

    def forward(self, images):
        # Dividing images into patches
        n, c, h, w = images.shape
        patches = patchify(images, self.n_patches).to(self.positional_embeddings.device)

        # Running linear layer tokenization
        # Map the vector corresponding to each patch to the hidden size dimension
        tokens = self.linear_mapper(patches)

        # Adding classification token to the tokens
        tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)

        # Adding positional embedding
        out = tokens + self.positional_embeddings.repeat(n, 1, 1)

        # Transformer Blocksx
        for block in self.blocks:
            out = block(out)

        # Getting the classification token only
        out = out[:, 0]

        return self.mlp(out) # Map to output dimension, output category distribution




In [ ]:

def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result



In [ ]:
image_dir = Path("/content/drive/MyDrive/Colab Notebooks/축산물이미지데이터와라벨정보_샘플데이터/원천데이터/1. 소도제/1.segmentation")

filepaths = pd.Series(sorted(list(image_dir.glob(r'**/*.jpg'))), name='Filepath').astype(str)
# print(filepaths[1])

def extract_id(filepath):
    segments = filepath.split('/')
    # Get the last segment
    filename = segments[-1]
    # Split the filename based on '_'
    filename_parts = filename.split('_')
    # Extract the ID from the filename
    id = filename_parts[-1].split('.')[0]
    return id

id = pd.Series(filepaths.map(extract_id), name='Id').astype(int)
print(id)


/content/drive/MyDrive/Colab Notebooks/축산물이미지데이터와라벨정보_샘플데이터/원천데이터/1. 소도제/1.segmentation/1/QC_cow_segmentation_1_000014.jpg
0          6
1         14
2         18
3         22
4         30
       ...  
995    60250
996    60252
997    60265
998    60270
999    60277
Name: Id, Length: 1000, dtype: int64


In [73]:
import json
from pandas import json_normalize

label_dir = Path("/content/drive/MyDrive/Colab Notebooks/축산물이미지데이터와라벨정보_샘플데이터/라벨링데이터/1.소도체")

# Get the file paths of JSON files in the directory
json_files = list(label_dir.glob("*.json"))

# Create an empty list to store the DataFrames
dfs = []

# Iterate over each JSON file
for file_path in json_files:
    # Read JSON data from the file
    with open(file_path) as file:
        json_data = json.load(file)

    # Extract the required information
    file_id = file_path.stem.split('_')[-1]  # Extract the last ID from the file name
    grade = json_data['label_info']['shapes'][0]['grade']
    gender = json_data['label_info']['shapes'][0]['gender']
    width = json_data['label_info']['image']['width']
    height = json_data['label_info']['image']['height']

    # Create a DataFrame
    df = pd.DataFrame({
        'Filepath': filepaths[filepaths.str.contains(file_id)],  # Filter the matching file path based on the file ID
        'grade': [grade],
        'gender': [gender],
        'width': [width],
        'height': [height]
    })

    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Print the combined DataFrame
print(combined_df)
print(combined_df.groupby('gender').count())
print(combined_df.groupby('grade').count())
print(combined_df.columns)

df = combined_df.drop(['grade', 'width', 'height'], inplace = True, axis = 1)
print(combined_df.columns)


                                              Filepath grade  gender  width  \
0    /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1  female   1080   
1    /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1   steer   1080   
2    /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1  female   1080   
3    /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1  female   1080   
4    /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1   steer   1080   
..                                                 ...   ...     ...    ...   
995  /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1   steer   1080   
996  /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1   steer   1080   
997  /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1   steer   1080   
998  /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1   steer   1080   
999  /content/drive/MyDrive/Colab Notebooks/축산ᄆ...     1  female   1080   

     height  
0      1920  
1      1920  
2      19

In [ ]:

def main():
    # Loading data
    transform = ToTensor()

    train_set = MNIST(root='./../datasets', train=True, download=True, transform=transform)
    test_set = MNIST(root='./../datasets', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
    test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

    # Defining model and training options
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")
    model = MyViT((1, 28, 28), n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10).to(device)
    N_EPOCHS = 5
    LR = 0.005

    # Training loop
    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()
    for epoch in trange(N_EPOCHS, desc="Training"):
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)

            train_loss += loss.detach().cpu().item() / len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")

    # Test loop
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0
        for batch in tqdm(test_loader, desc="Testing"):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            test_loss += loss.detach().cpu().item() / len(test_loader)

            correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
            total += len(x)
        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {correct / total * 100:.2f}%")

